# Voyages API Voyages Data Like Use Case

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VoyagesAPI/VoyagesAPI-VoyagesDataLike.ipynb). 

A Voyage is defined as a sequence of Load operations followed by a sequence of Discharges. Users of **Signal Ocean Platform** interface with the concept of a voyage in different levels of detail. For example in the Voyages tab of Vessels Data (https://app.signalocean.com/vessels) users can see  all the operations of a voyage even at jetty level.  
However very often arises the need of conducting an analysis of the voyages for a specific vessel class for a specific time window. This need is accommodated by the  **Voyages Data Dashboard** (https://app.signalocean.com/reportsindex/voyagesdatalive).  

The level of detail provided by the Voyages Data Dashboard has been tailored, having in mind the neccessary information needed to carry out such an analysis without being overwhelmed by the full data provided by Signal Ocean Platform regarding the voyages of the vessels.  

While both ```get_voyages``` and ``get_voyages_flat`` functions of the Signal SDK return the full low level data available, in this example we are going to construct a dataframe that resembles the form of ***Voyages Data Dashboard***

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

In [2]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime, timezone
import datetime as dt

In [3]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages

In [4]:
vlcc_id = 84
date_from = datetime.now(timezone.utc) - timedelta(days=365)

In [5]:
voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [6]:
voyages = pd.DataFrame(v.__dict__ for v in voyages)
events = pd.DataFrame(e.__dict__ for voyage_events in voyages['events'].dropna() for e in voyage_events)

In [7]:
def get_open_load_discharge_events(voyage_events):
    open_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Start'), None)
    load_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Load'), None)
    discharge_event = next((e.__dict__ for e in voyage_events or [] if e.purpose=='Discharge'), None)
    return pd.Series((open_event,load_event, discharge_event))
    
voyages[['open_event','load_event','discharge_event']] = voyages['events'].apply(get_open_load_discharge_events)

In [11]:
mapping_dict = {'port_name':['open_port','load_port','discharge_port'],
                'area_name_level0':['open_port_narrow_area','load_port_narrow_area','discharge_port_narrow_area'], 
                'area_name_level2':['open_port_wide_area','load_port_wide_area','discharge_port_wide_area'], 
                'area_name_level1':['open_port_area','load_port_area','discharge_port_area'], 
                'country':['open_port_country','load_port_country','discharge_port_country'],
                'sailing_date':['open_port_sailing_date','load_port_sailing_date','discharge_port_sailing_date'], 
                'longitude':['open_port_longitude','load_port_longitude','discharge_port_longitude'], 
                'latitude':['open_port_latitude','load_port_latitude','discharge_port_latitude'],
                'event_horizon':['open_horizon','load_horizon','discharge_horizon']}

events = {0:'open_event',1:'load_event',2:'discharge_event'}

In [12]:
for feature,targets in mapping_dict.items():
    for num,target in enumerate(targets):
        voyages[target] = voyages[events[num]].apply(lambda e: e[feature] if isinstance(e,dict) else None)

In [13]:
#since start_date is now in the top level of the voyages api response, in order for the data not to be duplicated 
# open_port_sailing_date is NaT in each row. We are going to resolve it with the following:

voyages['open_port_sailing_date'] = voyages.start_date

In [14]:
def get_last_discharge_port(voyage):
    return next((e.port_name for e in reversed(voyage['events'][:-2] or []) if e.purpose=='Discharge'), None)
    
voyages['last_discharge_port'] = voyages.apply(get_last_discharge_port, axis=1)

In [15]:
def get_last_3_months_ind(laycan_from):
    if not pd.isnull(laycan_from):
        laycan_from = pd.to_datetime(laycan_from)
        return 1 if ((laycan_from.date()-date.today()).days<4*30) else 0
    else:
        return 0
    
voyages['last_3_months_ind'] = voyages['laycan_from'].apply(get_last_3_months_ind)

In [16]:
def get_sts_load_ind(load_event):
    return next((1 for d in load_event["event_details"] or [] if d.event_detail_type =='StS'), 0)

def get_sts_discharge_ind(discharge_event):
    return next((1 for d in discharge_event["event_details"] or [] if d.event_detail_type =='StS'), 0)


voyages.loc[voyages.discharge_event.notna(),'sts_discharge_ind'] = \
voyages.loc[voyages.discharge_event.notna(),'discharge_event'].apply(get_sts_discharge_ind)
voyages.loc[voyages.load_event.notna(),'sts_load_ind'] = \
voyages.loc[voyages.load_event.notna(),'load_event'].apply(get_sts_load_ind)

In [17]:
vessel_status_dict = {
    1:"Voyage", 2:"Breaking", 3:"Domestic Trade", 4:"FPSO", 5:"FPSO Conversion", 
    6:"Inactive", 7:"Storage Vessel", 9:"Conversion"
}
voyages['vessel_status'] = voyages.vessel_status_id.replace(vessel_status_dict)

In [18]:
wanted_columns = ['imo',
                  'voyage_number',
                  'vessel_type_id',
                  'vessel_name',
                  'vessel_type',
                  'vessel_class',
                  'trade',
                  'commercial_operator',
                  'charterer',
                  'rate',
                  'rate_type',
                  'cargo_type',
                  'cargo_group',
                  'quantity',
                  'laycan_from',
                  'laycan_to',
                  'fixture_status_id',
                  'fixture_status',
                  'fixture_date',
                  'fixture_is_coa',
                  'fixture_is_hold',
                  'open_port',
                  'load_port',
                  'discharge_port',
                  'open_port_country',
                  'load_port_country',
                  'discharge_port_country',
                  'open_port_narrow_area',
                  'load_port_narrow_area',
                  'discharge_port_narrow_area',
                  'open_port_wide_area',
                  'load_port_wide_area',
                  'discharge_port_wide_area',
                  'open_port_area',
                  'load_port_area',
                  'discharge_port_area',
                  'open_port_sailing_date',
                  'load_port_sailing_date',
                  'discharge_port_sailing_date',
                  'open_port_longitude',
                  'load_port_longitude',
                  'discharge_port_longitude',
                  'open_port_latitude',
                  'load_port_latitude',
                  'discharge_port_latitude',
                  'sts_discharge_ind',
                  'sts_load_ind',
                  'load_horizon',
                  'discharge_horizon',
                  'ballast_distance',
                  'laden_distance']

voyages = voyages[wanted_columns]
voyages

,imo,voyage_number,vessel_type_id,vessel_name,vessel_type,vessel_class,trade,commercial_operator,charterer,rate,...,discharge_port_longitude,open_port_latitude,load_port_latitude,discharge_port_latitude,sts_discharge_ind,sts_load_ind,load_horizon,discharge_horizon,ballast_distance,laden_distance
0,8765216,1,1,Belanak Natuna,Tanker,VLCC,Crude,Unknown,None,NaN,...,NaN,-7.5515,NaN,NaN,NaN,NaN,None,None,1837.81,NaN
1,9079078,36,1,Navarz,Tanker,VLCC,Crude,NITC,None,NaN,...,NaN,-23.8000,NaN,NaN,NaN,NaN,None,None,87.90,NaN
2,9102241,126,1,Hapon,Tanker,VLCC,Crude,Bahri,None,NaN,...,121.9717,40.2106,1.2723,29.9473,0.0,1.0,Historical,Historical,2822.06,2278.43
3,9102241,127,1,Hapon,Tanker,VLCC,Crude,Bahri,None,NaN,...,121.1392,29.9473,1.2723,37.7139,0.0,0.0,Historical,Historical,2100.19,2859.31
4,9102241,128,1,Hapon,Tanker,VLCC,Crude,Bahri,None,NaN,...,119.7801,37.7139,1.4885,35.5704,0.0,0.0,Historical,Historical,2565.16,2662.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,9902615,1,1,Murban,Tanker,VLCC,Crude,ADNATCO,None,NaN,...,32.3875,38.9398,25.1475,29.5763,0.0,1.0,Current,Future,7270.01,33.95
3826,9902627,1,1,Cs Hunan Venture,Tanker,VLCC,Crude,Unknown,None,NaN,...,NaN,38.9028,NaN,NaN,NaN,NaN,None,None,675.82,NaN
3827,9904065,1,1,C.Grace,Tanker,VLCC,Crude,Hyundai,Total,2150000.0,...,0.1317,34.7333,24.1731,49.6654,0.0,0.0,Future,Future,6153.78,NaN
3828,9904077,1,1,C.Brave,Tanker,VLCC,Crude,Unknown,Adnoc,2300000.0,...,0.1317,34.7348,34.7651,49.6654,0.0,0.0,Future,Future,33.51,NaN


In [19]:
voyages = voyages.astype(
                        {
                            'laycan_from':'datetime64[ns]', 
                            'laycan_to':'datetime64[ns]',
                            'fixture_date':'datetime64[ns]',
                            'laycan_to':'datetime64[ns]',
                            'fixture_date':'datetime64[ns]',
                            'open_port_sailing_date':'datetime64[ns]',
                            'load_port_sailing_date':'datetime64[ns]',
                            'discharge_port_sailing_date':'datetime64[ns]'
                        })

In [20]:
voyages.to_excel('voyages_data.xlsx')